# Other Prices methods tutorial

#### Sections
* [`request_all_prices`](#request_all_prices)
* [`prices_for_symbols`](#prices_for_symbols)
* [`to_csv`](#to_csv)

#### Note

The cell **outputs** shown in this tutorial are based on executing the cells at the time shown in the output of the following cell. Simply rerun the cells to bring any dynamic output up to date.

In [2]:
import pandas as pd
from zoneinfo import ZoneInfo
now = pd.Timestamp.now(tz=ZoneInfo("UTC")).floor("T")
print(f"{now!r}")
print(f"{now.astimezone(ZoneInfo('America/New_York'))!r}")

Timestamp('2022-05-13 13:40:00+0000', tz='UTC')
Timestamp('2022-05-13 09:40:00-0400', tz='America/New_York')


## Setup

Run the following cell to import tutorial dependencies.

In [3]:
from market_prices import PricesYahoo
from market_prices.support import tutorial_helpers as th
from market_prices import helpers

Execute the following cell to define a `PricesYahoo` instance and a few variables that will be used to demonstrate the methods covered by this tutorial.

In [4]:
prices = PricesYahoo("MSFT, AZN.L, 9988.HK", lead_symbol="MSFT")
xnys = prices.calendars["MSFT"]
xhkg = prices.calendars["9988.HK"]
xlon = prices.calendars["AZN.L"]

H1_start, H1_end = th.get_sessions_range_for_bi(prices, prices.bis.H1)

## `request_all_prices`

Calling the Prices instance's `request_all_data` method will result in **all** currently available data being requested and stored locally. Consequently, further requests to the data provider will be limited to any 'live' prices that may have since changed or become available.

The method returns a dictionary that expresses, by base interval, the range over which prices are now stored locally.

In [5]:
# might take a little while
prices.request_all_prices()

{<BaseInterval.T1: Timedelta('0 days 00:01:00')>: [Interval('2022-04-13 13:42:00', '2022-05-13 13:15:00', closed='both')],
 <BaseInterval.T2: Timedelta('0 days 00:02:00')>: [Interval('2022-03-31 13:42:00', '2022-05-13 13:14:00', closed='both')],
 <BaseInterval.T5: Timedelta('0 days 00:05:00')>: [Interval('2022-03-14 13:45:00', '2022-05-13 13:15:00', closed='both')],
 <BaseInterval.H1: Timedelta('0 days 01:00:00')>: [],
 <BaseInterval.D1: Timedelta('1 days 00:00:00')>: [Interval('1986-03-13', '2022-05-12', closed='both')]}

Notice that no prices are stored for the H1 interval. This is because prices include an equity listed on the New York Stock Exchange and one listed on the London Stock Exchange. The opening times of these exchanges are such that sessions often overlap although the indices at H1 are not aligned. Consequently, it's not possible to evaluate common hourly indices (at least not using H1 data).

The rest of this section demonstrates some under-the-bonnet behaviour that can be safely skipped - move onto [`prices_for_symbols`](#prices_for_symbols) if you're not interested.

Srictly speaking, valid hourly data will be available for any session where only one of these two exchanges is open, or when one or both have irregular opening hours such that they do not overlap (or align if they do). If intraday prices are requested for just such a session then a request for hourly data will be sent to the provider if intraday prices for that session are not available at an interval smaller than one hour.

In [6]:
xnys_sessions = xnys.sessions_in_range(H1_start, H1_end)
xlon_sessions = xlon.sessions_in_range(H1_start, H1_end)
session = helpers.to_tz_naive(xnys_sessions.difference(xlon_sessions))[0]
session  # for reference

Timestamp('2020-08-31 00:00:00')

In [7]:
prices.get(start=session, end=session)

symbol                                     9988.HK                      AZN.L  \
                                             close high low open volume close   
[2020-08-31 09:30:00, 2020-08-31 10:30:00)     NaN  NaN NaN  NaN    NaN   NaN   
[2020-08-31 10:30:00, 2020-08-31 11:30:00)     NaN  NaN NaN  NaN    NaN   NaN   
[2020-08-31 11:30:00, 2020-08-31 12:30:00)     NaN  NaN NaN  NaN    NaN   NaN   
[2020-08-31 12:30:00, 2020-08-31 13:30:00)     NaN  NaN NaN  NaN    NaN   NaN   
[2020-08-31 13:30:00, 2020-08-31 14:30:00)     NaN  NaN NaN  NaN    NaN   NaN   
[2020-08-31 14:30:00, 2020-08-31 15:30:00)     NaN  NaN NaN  NaN    NaN   NaN   
[2020-08-31 15:30:00, 2020-08-31 16:30:00)     NaN  NaN NaN  NaN    NaN   NaN   

symbol                                                                 MSFT  \
                                           high low open volume       close   
[2020-08-31 09:30:00, 2020-08-31 10:30:00)  NaN NaN  NaN    NaN  226.350006   
[2020-08-31 10:30:00, 2020-08-31 11:30:00)  NaN NaN  NaN    NaN  224.936905   
[2020-08-31 11:30:00, 2020-08-31 12:30:00)  NaN NaN  NaN    NaN  225.595001   
[2020-08-31 12:30:00, 2020-08-31 13:30:00)  NaN NaN  NaN    NaN  225.249695   
[2020-08-31 13:30:00, 2020-08-31 14:30:00)  NaN NaN  NaN    NaN  225.690002   
[2020-08-31 14:30:00, 2020-08-31 15:30:00)  NaN NaN  NaN    NaN  225.764999   
[2020-08-31 15:30:00, 2020-08-31 16:30:00)  NaN NaN  NaN    NaN  225.059998   

symbol                                                              \
                                                  high         low   
[2020-08-31 09:30:00, 2020-08-31 10:30:00)  228.699997  226.169998   
[2020-08-31 10:30:00, 2020-08-31 11:30:00)  226.690002  224.580002   
[2020-08-31 11:30:00, 2020-08-31 12:30:00)  225.899994  224.309998   
[2020-08-31 12:30:00, 2020-08-31 13:30:00)  225.839996  225.190002   
[2020-08-31 13:30:00, 2020-08-31 14:30:00)  225.929993  225.169998   
[2020-08-31 14:30:00, 2020-08-31 15:30:00)  226.539993  225.449997   
[2020-08-31 15:30:00, 2020-08-31 16:30:00)  226.850006  225.000000   

symbol                                                           
                                                  open   volume  
[2020-08-31 09:30:00, 2020-08-31 10:30:00)  227.149994  7803484  
[2020-08-31 10:30:00, 2020-08-31 11:30:00)  226.380005  4225311  
[2020-08-31 11:30:00, 2020-08-31 12:30:00)  224.925003  2408480  
[2020-08-31 12:30:00, 2020-08-31 13:30:00)  225.589996  1467267  
[2020-08-31 13:30:00, 2020-08-31 14:30:00)  225.256897  1968181  
[2020-08-31 14:30:00, 2020-08-31 15:30:00)  225.699997  2293104  
[2020-08-31 15:30:00, 2020-08-31 16:30:00)  225.759995  3177857

The following internal attribute returns the dictionary that expresses the locally stored data. It shows that a little H1 data is indeed now stored locally.

In [8]:
prices._pdata_ranges

{<BaseInterval.T1: Timedelta('0 days 00:01:00')>: [Interval('2022-04-13 13:42:00', '2022-05-13 13:15:00', closed='both')],
 <BaseInterval.T2: Timedelta('0 days 00:02:00')>: [Interval('2022-03-31 13:42:00', '2022-05-13 13:14:00', closed='both')],
 <BaseInterval.T5: Timedelta('0 days 00:05:00')>: [Interval('2022-03-14 13:45:00', '2022-05-13 13:15:00', closed='both')],
 <BaseInterval.H1: Timedelta('0 days 01:00:00')>: [Interval('2020-08-31 13:30:00', '2020-08-31 20:30:00', closed='both')],
 <BaseInterval.D1: Timedelta('1 days 00:00:00')>: [Interval('1986-03-13', '2022-05-12', closed='both')]}

## `prices_for_symbols`
`prices_for_symbols` returns an instance of the Prices class for a specified subset of symbols. The advantage of using this method, as opposed to simply creating a new instance for the symbols, is that all previously requested data will be copied over (requests to the data provider for previously requested data will not be repeated).

In [9]:
new_prices = prices.prices_for_symbols("MSFT, 9988.HK")

In [10]:
new_prices.symbols

['MSFT', '9988.HK']

A little dig into the internals shows that the locally stored T5 data was copied over.

In [11]:
new_prices._pdata[new_prices.bis.T5]._table

symbol                                            MSFT              \
                                                  open        high   
[2022-03-14 13:45:00, 2022-03-14 13:50:00)  284.269989  284.329987   
[2022-03-14 13:50:00, 2022-03-14 13:55:00)  283.019989  283.699005   
[2022-03-14 13:55:00, 2022-03-14 14:00:00)  283.089996  283.929901   
[2022-03-14 14:00:00, 2022-03-14 14:05:00)  283.029999  283.565002   
[2022-03-14 14:05:00, 2022-03-14 14:10:00)  282.660004  284.209900   
...                                                ...         ...   
[2022-05-13 07:50:00, 2022-05-13 07:55:00)         NaN         NaN   
[2022-05-13 07:55:00, 2022-05-13 08:00:00)         NaN         NaN   
[2022-05-13 13:30:00, 2022-05-13 13:35:00)  257.440002  258.709900   
[2022-05-13 13:35:00, 2022-05-13 13:40:00)  255.755005  257.380005   
[2022-05-13 13:40:00, 2022-05-13 13:45:00)  256.839996  256.839996   

symbol                                                                         \
                                                   low       close     volume   
[2022-03-14 13:45:00, 2022-03-14 13:50:00)  282.480011  283.109985   593177.0   
[2022-03-14 13:50:00, 2022-03-14 13:55:00)  282.470001  283.109985   367264.0   
[2022-03-14 13:55:00, 2022-03-14 14:00:00)  282.600006  283.035004   565450.0   
[2022-03-14 14:00:00, 2022-03-14 14:05:00)  282.000000  282.659912   411206.0   
[2022-03-14 14:05:00, 2022-03-14 14:10:00)  282.500000  284.167603   273435.0   
...                                                ...         ...        ...   
[2022-05-13 07:50:00, 2022-05-13 07:55:00)         NaN         NaN        NaN   
[2022-05-13 07:55:00, 2022-05-13 08:00:00)         NaN         NaN        NaN   
[2022-05-13 13:30:00, 2022-05-13 13:35:00)  255.699997  255.710007  2388245.0   
[2022-05-13 13:35:00, 2022-05-13 13:40:00)  255.350006  256.839996   738217.0   
[2022-05-13 13:40:00, 2022-05-13 13:45:00)  256.839996  256.839996        0.0   

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-03-14 13:45:00, 2022-03-14 13:50:00)        NaN        NaN        NaN   
[2022-03-14 13:50:00, 2022-03-14 13:55:00)        NaN        NaN        NaN   
[2022-03-14 13:55:00, 2022-03-14 14:00:00)        NaN        NaN        NaN   
[2022-03-14 14:00:00, 2022-03-14 14:05:00)        NaN        NaN        NaN   
[2022-03-14 14:05:00, 2022-03-14 14:10:00)        NaN        NaN        NaN   
...                                               ...        ...        ...   
[2022-05-13 07:50:00, 2022-05-13 07:55:00)  82.449997  82.449997  82.349998   
[2022-05-13 07:55:00, 2022-05-13 08:00:00)  82.349998  82.500000  80.000000   
[2022-05-13 13:30:00, 2022-05-13 13:35:00)        NaN        NaN        NaN   
[2022-05-13 13:35:00, 2022-05-13 13:40:00)        NaN        NaN        NaN   
[2022-05-13 13:40:00, 2022-05-13 13:45:00)        NaN        NaN        NaN   

symbol                                                            
                                                close     volume  
[2022-03-14 13:45:00, 2022-03-14 13:50:00)        NaN        NaN  
[2022-03-14 13:50:00, 2022-03-14 13:55:00)        NaN        NaN  
[2022-03-14 13:55:00, 2022-03-14 14:00:00)        NaN        NaN  
[2022-03-14 14:00:00, 2022-03-14 14:05:00)        NaN        NaN  
[2022-03-14 14:05:00, 2022-03-14 14:10:00)        NaN        NaN  
...                                               ...        ...  
[2022-05-13 07:50:00, 2022-05-13 07:55:00)  82.400002  1091340.0  
[2022-05-13 07:55:00, 2022-05-13 08:00:00)  82.199997  1590700.0  
[2022-05-13 13:30:00, 2022-05-13 13:35:00)        NaN        NaN  
[2022-05-13 13:35:00, 2022-05-13 13:40:00)        NaN        NaN  
[2022-05-13 13:40:00, 2022-05-13 13:45:00)        NaN        NaN  

[5928 rows x 10 columns]

## `to_csv`
`to_csv` provides for exporting price data to .csv files. The method has one required argument 'path' which takes a path to an **existing** directory to which the .csv files should be written. If no further arguments are passed then the default implementation will export all available price data as one csv file per aligned base interval per symbol, such that if there are 3 base intervals and 5 symbols then 15 .csv files will be created (the base intervals for a prices instance can be inspected with `prices.bis.__members__`).

Optional arguments provide for defining the `intervals` for which price data should be exported together with which symbols to `include` or `exclude`. The period over which price data is to be exported and the configuration of that data can be defined by passing any kwargs that are accepted by the `get` method (with the exception of 'interval'). For example 'start', 'days', 'anchor', 'priority', 'strict' etc are all valid keyword arguments.

Files exported with `to_csv` can be retrieved with the default implementation of the `PricesCsv` class. (NB this requires that the exported data conforms with the requirements of the `PricesCsv` class, for example that prices are anchored on the 'open' and have an interval no higher than daily. Files exported with the default implementation will always be retrievable via the `PricesCsv` class.)

See the method doc for further information...

In [ ]:
prices.to_csv?

```
Signature:
prices.to_csv(
    path: 'Annotated[Union[str, Path], Coerce(Path), Parser(parsing.verify_directory)]',
    intervals: 'Optional[Union[str, pd.Timedelta, datetime.timedelta, list[str], list[pd.Timedelta], list[datetime.timedelta]]]' = None,
    include: 'Optional[mptypes.Symbols]' = None,
    exclude: 'Optional[mptypes.Symbols]' = None,
    **kwargs,
) -> 'list[Path]'
Docstring:
Export price data to .csv file(s).

Note: Exported price data can be retrieved with the default
implementation of the `PricesCsv` class (requires that the
exported data conforms with the requirements of the
`PricesCsv` class, for example that prices are anchored on
the 'open' and have an interval no higher than daily).

Price data will be exported by symbol by interval, such that if
data is requested for 3 intervals and 5 symbols then 15 .csv
files will be created.

.csv filenames will follow the format:
    <SYMBOL>_<INTERVAL>_<YYMMDD>_<YYMMDD>.csv
    For example:
        MSFT_5T_240122_240215.csv
    This file would hold '5T' (i.e. 5 minute) price data for
    the symbol MSFT covering the period from 2024-01-22 through
    2024-02-15. Note: for intraday intervals the dates will
    represent the earliest and latest sessions for which at least
    some price data is included.

Parameters
----------
path
    Directory to which .csv files should be written. This path
    must exist.

intervals
    Intervals for which price data is to be exported. To define
    a single interval pass as for the 'interval` parameter of the
    `.get` method. To define multiple intervals pass as a list
    of one of the types that's acceptable input to the 'interval`
    parameter of the `.get` method.

    By default (None) .csv files are exported for all available
    base intervals.

include : list[str] | str | None
    Symbol or symbols to include in export. All other symbols will
    be excluded. If passed, do not pass `exclude`.

    By default, if neither include nor exclude are passed then data
    will be exported for all symbols.

exclude : list[str] | str | None
    Symbol or symbols to exclude from export. Data will be exported
    for all other symbols. If passed, do not pass `include`.

    By default, if neither exclude nor include are passed then data
    will be exported for all symbols.

kwargs
    All other kwargs will be passed on to the `.get` method to
    define the period over which prices are to be exported. Can
    include other options, for example 'anchor', 'priority',
    'strict' etc.

    If no other kwargs are not passed then by default all available
    data will be exported for each requested symbol / interval.

Returns
-------
paths
    List of Path objects to which data exported.
```